In [13]:
#Device check and load model into device
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [14]:
! pip install thop

In [15]:
import pandas as pd
import os
import torch
import time
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torchvision.models as models
import matplotlib.pyplot as plt
from sklearn.metrics import *


##HYPER-PARAM
batch_size = 200
epochs = 120
max_lr = 0.001
grad_clip = 0.01
weight_decay =0.001
opt_func = torch.optim.Adam

##DOWNLOAD dataset
train_data = torchvision.datasets.CIFAR100('./', train=True, download=True)
# Stick all the images together to form a 1600000 X 32 X 3 array
x = np.concatenate([np.asarray(train_data[i][0]) for i in range(len(train_data))])
# calculate the mean and std along the (0, 1) axes
mean = np.mean(x, axis=(0, 1))/255
std = np.std(x, axis=(0, 1))/255
# the the mean and std
mean=mean.tolist()
std=std.tolist()

#随机遮挡：
class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
        	# (x,y)表示方形补丁的中心位置
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

##TRANSFORM
transform_train = tt.Compose([tt.RandomCrop(32, padding=4,padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         tt.Normalize(mean,std,inplace=True),
                         Cutout(n_holes=1, length=16)])
transform_test = tt.Compose([tt.ToTensor(), tt.Normalize(mean,std)])
##DATASET and DATALOADER
trainset = torchvision.datasets.CIFAR100("./",
                                         train=True,
                                         download=True,
                                         transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size, shuffle=True, num_workers=2,pin_memory=True)

testset = torchvision.datasets.CIFAR100("./",
                                        train=False,
                                        download=True,
                                        transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size*2,pin_memory=True, num_workers=2)
#LOADER
device = get_default_device()
trainloader = DeviceDataLoader(trainloader, device)
testloader = DeviceDataLoader(testloader, device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [16]:
cuda = torch.cuda.is_available()

In [17]:
##TRAINING SETUP
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

In [18]:
class BasicConv2d(nn.Module):

  def __init__(self, in_channels, out_channels, **kwargs):
    super(BasicConv2d, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
    self.bn = nn.BatchNorm2d(out_channels)

  def forward(self, x):
    x = self.conv(x)
    x = self.bn(x)
    #使用silu激活函数替代relu
    return F.silu(x, inplace=True)

In [19]:
class Inception3(ImageClassificationBase):
  def __init__(self, num_classes=100):
    super().__init__()
    
    self.Conv2d_1a_3x3 = BasicConv2d(3, 32, kernel_size=3, stride=1)
    self.Conv2d_2b_3x3 = BasicConv2d(32, 64, kernel_size=3, padding=1)
    self.Conv2d_3b_1x1 = BasicConv2d(64, 128, kernel_size=1)
    self.Conv2d_4a_3x3 = BasicConv2d(128, 192, kernel_size=3)
    self.Mixed_5b = InceptionA(192, pool_features=16)
    self.Mixed_5c = InceptionA(256, pool_features=48)
    self.Mixed_5d = InceptionA(288, pool_features=48)
    self.Mixed_6a = InceptionB(288)
    self.fc = nn.Sequential(nn.Linear(2032, 512),nn.Linear(512,num_classes))
    
  def forward(self, x):
    #先进行四层卷积
    x = self.Conv2d_1a_3x3(x)
    x = self.Conv2d_2b_3x3(x)
    x = self.Conv2d_3b_1x1(x)
    x = self.Conv2d_4a_3x3(x)
    #使用inceptionA模块训练三次
    x = F.max_pool2d(x, kernel_size=3, stride=2)
    x = self.Mixed_5b(x)
    x = self.Mixed_5c(x)
    x = self.Mixed_5d(x)
    #使用inceptionB模块训练一次
    x = self.Mixed_6a(x)
    x = F.max_pool2d(x, kernel_size=3, stride=2)
    x = F.dropout(x, p=0.1, training=self.training)
    x = torch.flatten(x, 1)
    x = self.fc(x)
    return x


class InceptionA(ImageClassificationBase):
  def __init__(self, in_channels, pool_features):
    super().__init__()
    self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size=1)
    
    self.branch5x5_1 = BasicConv2d(in_channels, 32, kernel_size=1)
    self.branch5x5_2 = BasicConv2d(32, 64, kernel_size=5, padding=2)
    
    self.branch3x3dbl_1 = BasicConv2d(in_channels, 32, kernel_size=1)
    self.branch3x3dbl_2 = BasicConv2d(32, 112, kernel_size=3, padding=1)
    
    self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size=1)

  def forward(self, x):
    #第一支路：1*1卷积核压缩通道数
    branch1x1 = self.branch1x1(x)
    #第二支路：1*1卷积核压缩通道数
    #        5*5卷积核获取特征
    branch5x5 = self.branch5x5_1(x)
    branch5x5 = self.branch5x5_2(branch5x5)
    #第三支路：1*1卷积核压缩通道数
    #        3*3卷积核获取特征
    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    #第四支路：承接最大池化步骤，1*1卷积核提取特征
    branch_pool = F.max_pool2d(x, kernel_size=3, stride=1, padding=1)
    branch_pool = self.branch_pool(branch_pool)

    outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
    return torch.cat(outputs, 1)


class InceptionB(ImageClassificationBase):

  def __init__(self, in_channels):
    super(InceptionB, self).__init__()
    self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size=1)
    self.branch3x3 = BasicConv2d(64, 124, kernel_size=3, stride=2)
    
    self.branch3x3dbl_1 = BasicConv2d(in_channels, 32, kernel_size=1)
    self.branch3x3dbl_2 = BasicConv2d(32, 96, kernel_size=3, stride=2)

  def forward(self, x):
    #第一支路：1*1卷积核压缩通道数
    #        3*3卷积核提取特征
    branch1x1 = self.branch1x1(x)
    branch3x3 = self.branch3x3(branch1x1)
    #第二支路：1*1卷积核压缩通道数
    #        3*3卷积核提取特征
    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    #第三支路：最大池化
    branch_pool = F.max_pool2d(x, kernel_size=3, stride=2)

    outputs = [branch3x3, branch3x3dbl, branch_pool]
    return torch.cat(outputs, 1)

model = Inception3().cuda()

In [20]:
###########mixup:
alpha = 1.0  # 默认设置为1
criterion = nn.CrossEntropyLoss()
for (inputs, labels) in trainloader:
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(inputs.size(0))
    images_a, images_b = inputs, inputs[index]
    labels_a, labels_b = labels, labels[index]
    mixed_images = lam * images_a + (1 - lam) * images_b
    outputs = model(mixed_images)
    _, preds = torch.max(outputs, 1)
    loss = lam * criterion(outputs, labels_a) + (1 - lam) * criterion(outputs, labels_b)

In [21]:
#Training Setup
@torch.no_grad()
def evaluate(model, test_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in test_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, test_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, test_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [22]:
#max_lr = 0.001
#Training(Using Multi_LR)
history = [evaluate(model, testloader)] ## Initial evaluation
# Fitting the first 1/4 
current_time=time.time()
history += fit_one_cycle(120, 0.0008, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
'''
torch.save(model.state_dict(), 'group22_pretrained_model.h5')
# Fitting the first 2/4 epochs
history += fit_one_cycle(20, max_lr/50, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
# Fitting the first 3/4 
history += fit_one_cycle(10, max_lr/100, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
# Fitting the first 4/4 epochs
history += fit_one_cycle(10 , max_lr/100, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
'''
# Print training time
time_train = time.time() - current_time
print('Training time: {:.2f} s'.format(time_train))

Epoch [0], last_lr: 0.00003, train_loss: 4.2964, val_loss: 3.8880, val_acc: 0.1268
Epoch [1], last_lr: 0.00004, train_loss: 3.8093, val_loss: 3.4850, val_acc: 0.1778
Epoch [2], last_lr: 0.00005, train_loss: 3.5319, val_loss: 3.2623, val_acc: 0.2087
Epoch [3], last_lr: 0.00006, train_loss: 3.3373, val_loss: 3.0281, val_acc: 0.2566
Epoch [4], last_lr: 0.00007, train_loss: 3.1578, val_loss: 2.8604, val_acc: 0.2873
Epoch [5], last_lr: 0.00008, train_loss: 2.9971, val_loss: 2.6836, val_acc: 0.3127
Epoch [6], last_lr: 0.00010, train_loss: 2.8491, val_loss: 2.5179, val_acc: 0.3525
Epoch [7], last_lr: 0.00012, train_loss: 2.7277, val_loss: 2.4143, val_acc: 0.3756
Epoch [8], last_lr: 0.00014, train_loss: 2.6279, val_loss: 2.3467, val_acc: 0.3922
Epoch [9], last_lr: 0.00017, train_loss: 2.5346, val_loss: 2.2307, val_acc: 0.4141
Epoch [10], last_lr: 0.00020, train_loss: 2.4287, val_loss: 2.1793, val_acc: 0.4283
Epoch [11], last_lr: 0.00022, train_loss: 2.3508, val_loss: 2.0609, val_acc: 0.4538
Ep

# Fitting the first 3/4 
history += fit_one_cycle(int(epochs/6), max_lr/100, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
# Fitting the first 4/4 epochs
history += fit_one_cycle(int(epochs/6), max_lr/100, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [23]:
torch.save(model.state_dict(), 'group22_pretrained_model1.h5')
# Generate testing accuracy, predicted label, confusion matrix, and table for classification report
def test_label_predictions(model, device, test_loader):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            prediction = output.argmax(dim=1, keepdim=True)
            actuals.extend(target.view_as(prediction))
            predictions.extend(prediction)
    return [i.item() for i in actuals], [i.item() for i in predictions]

y_test, y_pred = test_label_predictions(model, device, testloader)
cm=confusion_matrix(y_test, y_pred)
cr=classification_report(y_test, y_pred)
fs=f1_score(y_test,y_pred,average='weighted')
rs=recall_score(y_test, y_pred,average='weighted')
accuracy=accuracy_score(y_test, y_pred)
print('Confusion matrix:')
print(cm)
print(cr)
print('F1 score: %f' % fs)
print('Recall score: %f' % rs)
print('Accuracy score: %f' % accuracy)

Confusion matrix:
[[93  0  0 ...  0  0  0]
 [ 0 86  0 ...  0  0  0]
 [ 0  0 64 ...  0  3  0]
 ...
 [ 0  0  0 ... 82  0  0]
 [ 0  0  6 ...  0 57  0]
 [ 0  0  0 ...  0  0 70]]
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       100
           1       0.88      0.86      0.87       100
           2       0.64      0.64      0.64       100
           3       0.59      0.62      0.60       100
           4       0.61      0.60      0.60       100
           5       0.81      0.73      0.77       100
           6       0.79      0.79      0.79       100
           7       0.82      0.79      0.81       100
           8       0.92      0.90      0.91       100
           9       0.81      0.87      0.84       100
          10       0.57      0.60      0.59       100
          11       0.54      0.50      0.52       100
          12       0.78      0.77      0.77       100
          13       0.72      0.63      0.67       100
          14   

In [24]:
### Paramater Size and FLOPS
from thop import profile
input = torch.randn(1, 3, 32, 32)
input = input.to(device)
flops, params = profile(model, inputs=(input,))
print(flops)
print(params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
272840896.0
1846616.0
